In [3]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans

In [11]:
# Preprocessing

temp = ['Bowlegs', 'Knock Knees', 'Normal Knee']

for i in temp:
    k = 0
    for j in os.listdir('./train/Original/' + i):
        org = cv2.imread('./train/Original/' + i + '/' + j)
        gray = cv2.cvtColor(org, cv2.COLOR_BGR2GRAY)
        pr = cv2.resize(gray, (400, 400))

        cv2.imwrite('./train/Preprocessed/' + i + '/' + str(k) + '.jpg', pr)
        k += 1

In [12]:

input0 = './train/Preprocessed/'

for i in temp:
    for filename in os.listdir(input0 + i):
        img = cv2.resize(cv2.imread(input0 + i + '/' + filename), (512, 512))#resize image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        sift_image = cv2.drawKeypoints(gray, keypoints, img)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        #append to the csv file
        csv_data=out.to_csv('SIFT_' + i + '.csv', mode='a', index=False)

In [14]:
data1 = pd.read_csv('./SIFT_Bowlegs.csv', dtype='uint8')
data2 = pd.read_csv('./SIFT_Knock Knees.csv', dtype='uint8')
data3 = pd.read_csv('./SIFT_Normal Knee.csv', dtype='uint8')

data1 = data1.astype('uint8')
data2 = data2.astype('uint8')
data3 = data3.astype('uint8')

In [15]:
#performing kmeans on each class
#apple
kmeans1 = KMeans(n_clusters=5)
kmeans1.fit(data1)

#mango
kmeans2 = KMeans(n_clusters=5)
kmeans2.fit(data2)

#papaya
kmeans3 = KMeans(n_clusters=5)
kmeans3.fit(data3)

KMeans(n_clusters=5)

In [16]:
hist1=np.histogram(kmeans1.labels_,bins=[0,1,2,3,4,5])
hist2=np.histogram(kmeans2.labels_,bins=[0,1,2,3,4,5])
hist3=np.histogram(kmeans3.labels_,bins=[0,1,2,3,4,5])

print('histogram of bowed')
print(hist1,"\n")

print('histogram of knocked')
print(hist2,"\n")

print('histogram of normal')
print(hist3,"\n")

histogram of bowed
(array([3837, 4362, 2702, 3146, 3793], dtype=int64), array([0, 1, 2, 3, 4, 5])) 

histogram of knocked
(array([24509,  7626,  5143, 17287, 14957], dtype=int64), array([0, 1, 2, 3, 4, 5])) 

histogram of normal
(array([11380, 19924,  7529, 11657, 14588], dtype=int64), array([0, 1, 2, 3, 4, 5])) 



In [ ]:
#performing kmeans prediction of the entire apple dataset with the pretrained kmeans model
#initialising i=0; as its the first class
i=0
for j in temp:
    data=[]
    for filename in os.listdir(input0 + j):
        path = input0 + j + '/' + filename
        a=cv2.imread(path)
        resize=(512,512)
        img=cv2.resize(a,resize)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #initialise sift descriptor
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        #convert the descriptor array into a dataframe format
        out=pd.DataFrame(descriptors)
        
        #predict values of feature vector with pretrained kmeans
        #ValueError: Buffer dtype mismatch, expected 'float' but got 'double', in order to avoid this dtype=np.double
        array_double = np.array(out, dtype=np.double)

        a=kmeans1.predict(array_double)
        hist=np.histogram(a,bins=[0,1,2,3,4,5])
        #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
        data.append(hist[0])
    
    #convert Array to Dataframe and append to the list
    Output = pd.DataFrame(data)
    #add row class 
    Output["Class"] = i 
    csv_data=Output.to_csv('SIFT_'+ j + 'Final.csv', mode='a', index=False)
    i += 1

In [18]:
final = pd.read_csv('./SIFT_BowlegsFinal.csv')
temp1 = final.append(pd.read_csv('./SIFT_Knock KneesFinal.csv'))
tc = pd.read_csv('./SIFT_Normal KneeFinal.csv')
tcc = temp1.append(tc)

csv_data = tcc.to_csv('SIFT_FeatureFinal.csv', mode='a', index=False)
print(tcc)


      0   1   2   3   4  Class
0     3  21   4   8   2      0
1     5  16   4   7   3      0
2     4  54  10  14   9      0
3    34  79  29  25  36      0
4    33  73  33  17  34      0
..   ..  ..  ..  ..  ..    ...
244  22  30  11  39   9      2
245   3  24   1  14   2      2
246   3  26   4  15   3      2
247   4  25   3  21   3      2
248   5  32   7  23   4      2

[609 rows x 6 columns]


C:\Users\sumit\AppData\Local\Temp\ipykernel_14072\919743300.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp1 = final.append(pd.read_csv('./SIFT_Knock KneesFinal.csv'))
C:\Users\sumit\AppData\Local\Temp\ipykernel_14072\919743300.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tcc = temp1.append(tc)


In [5]:
df = pd.read_csv("./SIFT_FeatureFinal.csv")
df

,0,1,2,3,4,Class
0,3,5,24,4,2,0
1,8,3,18,5,1,0
2,8,15,28,6,34,0
3,38,39,60,14,52,0
4,41,33,69,8,39,0
...,...,...,...,...,...,...
604,28,11,30,37,5,2
605,4,1,16,7,16,2
606,4,2,18,9,18,2
607,5,1,15,12,23,2


In [11]:
df1 = df.iloc[:, 5:6]
df1

,Class
0,0
1,0
2,0
3,0
4,0
...,...
604,2
605,2
606,2
607,2
